In [ ]:
from scripts.utils import run_dbt_shell_command

run_dbt_shell_command("dbt parse")


In [ ]:
from dbt.cli.main import dbtRunner, dbtRunnerResult 
import logging

def run_dbt_command(
    dbt_command: str,
):
    
    res = dbtRunner().invoke(dbt_command.split(" ")[1:])

    if res.exception:
        raise RuntimeError("dbt command did not complete successfully.")
    
run_dbt_command("dbt parse")

In [ ]:
res = dbtRunner().invoke(["parse"])
res

In [5]:
import argparse
import logging
import os

from retry import retry
from scripts.utils import (
    ManifestInitRunError,
    call_github_api,
    download_manifest_json,
    run_dbt_command,
    send_github_pr_comment,
    set_logging_options,
)

env = "stg"

modified_nodes_raw = run_dbt_command(
    f"dbt ls --select state:modified,package:beyond_basics --state ./.state --resource-type model --target {env}"
)
print(f"{modified_nodes_raw=}")


08:44:08  Running with dbt=1.6.5
08:44:08  Registered adapter: bigquery=1.6.6
08:44:08  Found 8 models, 3 seeds, 32 tests, 1 source, 6 exposures, 0 metrics, 396 macros, 0 groups, 0 semantic models
08:44:08  The selection criterion 'state:modified,package:beyond_basics' does not match any nodes
08:44:08  No nodes selected!
modified_nodes_raw=[]


In [ ]:
import json
import logging
import os
import re
from datetime import datetime
from pathlib import Path
from typing import Any, Mapping, Optional, Union

import requests
from dbt.cli.main import dbtRunner
from google.cloud import bigquery, storage
from google.oauth2 import service_account
from retry import retry


res = dbtRunner().invoke( f"dbt ls --select state:modified,package:beyond_basics --state ./.state --resource-type model --target {env}".split(" ")[1:])

In [ ]:

# inspect the results
for r in res.result:
    print(r)